# より深いネットワークへ
ここでは、これまでに学んだ技術を使って、ディープなネットワークを作り、MNISTデータセットの手書き文字の認識に挑む。

## よりディープなネットワークとは
より深いCNNを、VGGと呼ぶ。ここでは、下図のような構成のネットワークを作成する。また、このネットワークの特徴としては、以下が挙げられる。

- 3×3の小さなフィルターによる畳み込み層
- 活性化関数はReLU
- 全結合層の後にDropoutレイヤを使用
- Adamによる最適化
- 重みの初期値として「Heの初期値」を使用

<img src='https://s3-ap-northeast-1.amazonaws.com/dragonarrow/uploads%2F1566104240795-VGG.png'>

このネットワークを、学習済みのデータを利用して実行して実行したのが以下である。

In [1]:
#ネットワークの実装
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *

class DeepConvNet:
    """認識率99%以上の高精度なConvNet

    ネットワーク構成は下記の通り
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 重みの初期化===========
        # 各層のニューロンひとつあたりが、前層のニューロンといくつのつながりがあるか（TODO:自動で計算する）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [9]:
#学習済みデータの精度を算出
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()

network.load_params(file_name="deep_convnet_params.pkl")

network.accuracy(x_test,t_test)

0.9935

99.35%と、高い認識精度が出ていることがわかる。

## さらに認識精度を高めるには
「<a href="https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html">What is the class of this image?</a>」というWebサイトには、様々なデータセットを対象に、これまで論文などで発表された手法の認識精度がランキング形式で掲載されている。MNISTデータセットの識別に関しては、このサイトで上位を占めているほとんどの手法はCNNをベースとしたものである。また、このランキングの上位の手法を参考にすると、様々な認識精度を高めるためのテクニックを学ぶことができる。例としては、アンサンブル学習、学習係数の減衰、**Data Augmation**(データの拡張)などがある。特にData Augmationは、簡単で有効な手法である。Data Augmationでは、入力画像をアルゴリズムによって回転や移動などの微小な変化を与えることによって人工的にデータを拡張する。データの数が限られている場合には特に有効である。

## 層を深くするということ
層を深くすることの重要性については、理論的にそれほど多くのことは分かっていない。しかし、今までの研究の結果などからデータ的には多少の説明ができる。まず、ILSVRCなどに代表される大規模画像認識のコンペティションでは、最近の多くの上位の手法は、層を深くする傾向にある。これは、層を深くすることによって認識精度を向上させることができるからだと読み取れる。また、層を深くすることで、ネットワークのパラメータ数を少なくすることができる。小さなフィルターを重ねることで、受容野を広くすることができるという利点がある。（受容野とは、ニューロンに変化を生じさせる局所的な空間領域のこと。）また、層を深くすることで、学習データを少なくすることができ、学習効率が上がる。（より複雑な特徴をとらえることができるため。）

# ディープラーニングの歴史
近年のディープラーニングについて紹介する。

## ImageNet
ImageNetは100万を超える画像データのセット。この巨大なデータセットを用いて、ILSVRCという画像認識のコンペティションが毎年行われている。この中では、様々なテスト項目があるが、「クラス分類」についての結果を見ると、2012年からは常にディープラーニングがトップとなっている。特に、2015年の150層超から構成されるResNetにおいては、誤認識率を3.5％まで抑え、人間を超えたとまで言われている。
そのようなディープラーニングの中でも、有名であるVGG、GoogLeNet、ResNetについて順に説明する。

## VGG
VGGは重みのある層を全部で16層または19層重ねて深くした、基本的なCNNである。

## GoogLeNet
GoogLeNetは以下のような構成のネットワークであり、特徴としては、ネットワークが横方向にも広がりをもっていることが挙げられる。
<img src='https://camo.qiitausercontent.com/ff9fdb708af0f67c865b0b1e41c4d0f42f0be738/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3230393730352f36376639383231332d613866662d646535322d326437652d3138613365663337636464322e706e67'>
<br>
横方向に幅を持つ構造をインセプション構造と言い、以下のような構成をしている。
<img src='https://miro.medium.com/max/1400/1*DKjGRDd_lJeUfVlY50ojOA.png'>
<br>
これは、サイズの異なるフィルターとプーリングを複数適用し、その結果を結合するもの。また、このネットワークでは、1×1のフィルターの畳み込み演算を使用することで、チャンネル方向にサイズを減らし、パラメータの削減や処理の高速化を実現した。

## ResNet
ResNetはMicrosoftのチームによって開発されたネットワーク。このネットワークでは、スキップ構造というものを用いて層を深くすることで性能が向上するようにできた。スキップ構造はとは、入力データを畳み込み層をまたいで出力に合算する構造。スキップ構造によって逆伝播時に信号が減衰することなく、効率のよい学習を行うことができる。ResNetでは2層ごとにスキップ構造を用いて入力と出力をつなぎ、層を深くしていく。実験によって、この方法で150層以上にしても認識精度は向上し続けることが分かった。

# ディープラーニングの高速化
現在、多くのディープラーニングのフレームワークはGPUによる高速な演算に対応している。また、複数のGPUやコンピュータで処理を行う分散学習にも対応していることがある。

## 高速化に関する問題
AlexNetでは、多くの時間が畳み込み層の処理に費やされる。その処理時間は、CPUでは89%、GPUでは95%にまで達する。この数値は推論時のものであるが、学習時にも時間がかかるのは畳み込み層での処理である。よって、この処理時間をいかに短縮するかというのが課題となる。

## GPUによる高速化
GPUはグラフィック専用ボードであるが、並列的な数値計算にも利用できる。そのような利用を、**GPUコンピューティング**という。現状、多くのディープラーニングのフレームワークが対応しているのは、NVIDIA社のGPUのみの場合が多い。

## 分散学習
複数のGPUやマシンで分散して計算を行い、学習することを分散学習という。TensorFlowやCNTKといったフレームワークは、分散学習を意識したものとなっている。分散学習においては、どのように計算を分散させるかということが問題とされている。

## 演算精度のビット削減
ディープラーニングの高速化においては、計算量に加えてハードウェア的条件がボトルネックとなりえる。このことから、ネットワークを流れるでデータのビット数はできるだけ小さくすることが望まれる。ディープラーニングにおいては、数値精度のビット数をそこまで必要としないということが研究で分かっている。これはニューラルネットワークのロバスト性によるものである。よって、ディープラーニングにでは16bitの半精度浮動小数点数（half float）を用いる。なお、Pythonでは一般的に64bitの浮動小数点数が使用される。また、最近では重みや中間データを1bitで表現する**Binarized Neural Networks**という手法が提案されている。

# ディープラーニングの実用例

## 物体検出
物体検出は、画像中から物体の位置の特定を含めてクラス分類を行う問題。この問題に対してCNNをベースとした手法がいくつか提案されている。その中でも、R-CNNと呼ばれるものの処理フローは以下のようなものである。
<img src='https://takaolab.com/wp-content/uploads/2018/11/r-cnn.png'>
<br>
2.Extract region proposalsにおいて、オブジェクトらしい領域（候補領域）を何らかの形で抽出し、それをCNNネットワークにかけて分類を行っている。また、「Faster R-CNN」という手法では、候補領域の抽出もCNNで行う。全ての処理を一つのCNNで行うため、処理速度の向上が期待できる。

## セグメンテーション
セグメンテーションは、画像に対してピクセルレベルでクラス分類を行う問題。しかし、全てのピクセルに対して推論処理を実行すると時間がかかるため、FCN(Fully Convolutional Network)という手法が提案された。これは、一般のCNNが全結合層を含むのに対し、全結合層を同じ働きをする畳み込み層に置き換えるもの。

## 画像キャプション作成
画像を与えると、その画像を説明する文章を自動で生成するもの。この問題における手法には、代表的なものとしてNICがある。NICはCNNとRNN(Recurrent Neural Network)から構成される。RNNは、言語や時系列データなどの連続性のあるデータによく用いられる。なお、画像と自然言語といったような複数の種類の情報を組み合わせて処理することを**マルチモーダル処理**といい、近年注目を集めている。

# ディープラーニングの未来
ディープラーニングの可能性を感じる研究をいくつか紹介する。

## 画像スタイル変換
ディープラーニングを用いて、アーティストのような絵を描かせるという研究が近年進んでいる。例えば、コンテンツ画像とスタイル画像の2つの画像を入力し、後者の描画スタイルを前者の画像に適用したものを出力するというものがある。

## 画像生成
何の画像も入力せず、新たな画像を描き出すというもの。これは、今までの教師あり学習と異なる、教師なし学習である。

## 自動運転
人間の代わりにコンピュータが自動車を運転するというもの。様々な環境でもロバストに走行領域を正しく認識する方法として、CNNベースの手法が期待されている。

## Deep Q-Network(強化学習)
人が試行錯誤を経て学ぶように、コンピュータにも試行錯誤させて学習させる方法を強化学習と呼ぶ。強化学習では、エージェントというものが環境に応じて行動を取り、その行動によって環境が変化するということを繰り返している。このとき、環境の変化によってエージェントは何らかの報酬を得る。この報酬を最大にするのが強化学習の目的である。ディープラーニングを用いた強化学習の手法として、Deep Q-Network（通称DQN）というものがある。これはQ学習という強化学習のアルゴリズムをベースとしたもの。